# TP3: Problemas NP-Completos

### ¿Hitting-Set Problem está en NP?

In [6]:
#Verificamos la solucion polinomialmente
def verificar_solucion(B, C):
    satisface_Bi = [False] * len(B)

    for c in C:                      # O(#C) 
        for i in range(len(B)):
            if c in B[i]:            # O(#B[i])
                satisface_Bi[i] = True

    return all(satisface_Bi)      # O(#B)         

### Funciones auxiliares


In [7]:
# Recibe el path con el archivo a leer.
# Leemos y devolvemos dos arrays. 
# - El primero corresponde a un array de listas, donde cada lista posee los jugadores que piden los diferentes medios y periodistas.
# - El segundo array posee todos los jugadores que aparecen en el array anterior, siendo los jugadores que Scaloni puede elegir, su preselección.
# Este algoritmo, devuelve una preselección sin orden.
def obtener_preseleccion_y_pedidos_de_la_prensa(path):
    with open(path, 'r') as archivo:
        lineas = archivo.readlines()

    pedidos_de_la_prensa = []
    preseleccion = set()

    for linea in lineas:
        preseleccion_linea = linea.strip().split(',')
        pedidos_de_la_prensa.append(preseleccion_linea)
        preseleccion.update(set(preseleccion_linea))

    return pedidos_de_la_prensa, list(preseleccion)


In [8]:
# Me guardo los resultados esperados, dados por la cátedra
class Resultado:
    def __init__(self, nombre_archivo, cantidad_minima, jugadores):
        self.nombre_archivo = nombre_archivo
        self.cantidad_minima = cantidad_minima
        self.jugadores = jugadores

def obtener_resultados_esperados(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        contenido = archivo.read() 
    resultados = []

    # Separa el contenido en bloques
    bloques = contenido.split('\n\n')
    
    # Itera a través de cada bloque y busca "Ganancia maxima" y "Plan de entrenamiento" dentro de cada uno
    for bloque in bloques:
        nombre_archivo = ''
        cantidad_minima = 0
        jugadores = []

        lineas = bloque.split('\n')
        for linea in lineas:
            if ".txt" in linea:
                nombre_archivo = linea
            if 'Cantidad' in linea:
                cantidad_minima = int(linea.split(":")[1].split("(")[0].strip())
                jugadores_linea = linea.split("(")[1].split(")")[0]
                jugadores = [jugador.strip() for jugador in jugadores_linea.split(",")]
                resultados.append(Resultado(nombre_archivo, cantidad_minima, jugadores))

    return resultados


### Sets de datos

In [5]:
#Paths de los archivos de la catedra
path_5 = 'sets/sets de la catedra/5.txt'
path_7 = 'sets/sets de la catedra/7.txt'
path_10_pocos = 'sets/sets de la catedra/10_pocos.txt'
path_10_todos = 'sets/sets de la catedra/10_todos.txt'
path_10_varios = 'sets/sets de la catedra/10_varios.txt'
path_15 = 'sets/sets de la catedra/15.txt'
path_20 = 'sets/sets de la catedra/20.txt'
path_75 = 'sets/sets de la catedra/75.txt'
path_100 = 'sets/sets de la catedra/100.txt'
path_200 = 'sets/sets de la catedra/200.txt'
resultados_sets_catedra = 'sets/sets de la catedra/Resultados Esperados.txt'

#### Creación sets de datos propios

In [18]:
import random

random.seed(0)

pedidos_de_prensa_max, _ = obtener_preseleccion_y_pedidos_de_la_prensa(path_200)
universe = list(set([pedido for pedidos in pedidos_de_prensa_max for pedido in pedidos]))

def generar_instancia(universe_size, num_sets, max_set_size):
    sets = [{universe[random.randint(1, universe_size)] for _ in range(random.randint(1, max_set_size))} for _ in range(num_sets)]
    return sets

sets_5 = generar_instancia(30, 5, 6)
sets_15 = generar_instancia(30, 15, 15)
sets_15_un_elemento = generar_instancia(30, 15, 1)
sets_20 = generar_instancia(30, 20, 6)
sets_50 = generar_instancia(30, 50, 2)

def crear_archivos(sets, nombre_archivo):
    with open(nombre_archivo, 'w') as archivo:
        for set in sets:
            archivo.write(','.join(set) + '\n')

crear_archivos(sets_5, 'sets/nuestros/5.txt')
crear_archivos(sets_15, 'sets/nuestros/15.txt')
crear_archivos(sets_15_un_elemento, 'sets/nuestros/15_un_elemento.txt')
crear_archivos(sets_20, 'sets/nuestros/20.txt')
crear_archivos(sets_50, 'sets/nuestros/50.txt')


In [19]:
#Paths de los archivos nuestros
path_5_nuestro = 'sets/nuestros/5.txt'
path_15_nuestro = 'sets/nuestros/15.txt'
path_15_un_elemento_nuestro = 'sets/nuestros/15_un_elemento.txt'
path_20_nuestro = 'sets/nuestros/20.txt'
path_50_nuestro = 'sets/nuestros/50.txt'

### Función verificadora

In [24]:
#Tomamos como base la verificación del Hitting-Set Problem. Y lo ajustamos para nuestro problema.
def verificar_solucion_para_Scaloni(B, C):
    if len(C) == 0:
        return False
    
    satisface_Bi = [False] * len(B)

    for c in C:                      # O(#C) 
        for i in range(len(B)):
            if c in B[i]:            # O(#B[i])
                satisface_Bi[i] = True

    return all(satisface_Bi)      # O(#B) 

### Solución por Backtracking


In [25]:
def conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, indice, solucion, max_jugadores):
    cantidad_de_jugadores = len(solucion_actual)
    
    if(cantidad_de_jugadores >= len(pedidos_de_prensa) or cantidad_de_jugadores >= max_jugadores):
        return 
    
    for i in range(indice, (len(preseleccion)-1)):
        solucion_actual.append(preseleccion[(i)])
        if verificar_solucion_para_Scaloni(pedidos_de_prensa, solucion_actual):
            if len(solucion_actual) < len(solucion) or len(solucion) == 0:
                solucion[:] = solucion_actual
        conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, i + 1, solucion, max_jugadores)
        solucion_actual.remove(solucion_actual[len(solucion_actual) - 1])



def solucion_para_Scaloni(preseleccion, pedidos_de_prensa):
    solucion = []
    solucion_actual = []
    max_jugadores = len(set([pedido for pedidos in pedidos_de_prensa for pedido in pedidos])) # ver complejidad
    conjunto_minimo_para_Scaloni(preseleccion, solucion_actual, pedidos_de_prensa, 0, solucion, max_jugadores)
    return solucion


# Ejemplo de uso
archivo_path = path_7
pedidos_de_la_prensa, preseleccion = obtener_preseleccion_y_pedidos_de_la_prensa(archivo_path)
solucion = solucion_para_Scaloni(preseleccion, pedidos_de_la_prensa)
print(solucion)

['Colidio', "Barcon't"]


#### Chequeo solución 

##### 1 - Usando sets de la cátedra

In [30]:
# correr pip install tabulate en consola para poder usar esta libreria 
from tabulate import tabulate

In [ ]:
pedidos_de_la_prensa_5, preseleccion_5 = obtener_preseleccion_y_pedidos_de_la_prensa(path_5)
solucion_5 = solucion_para_Scaloni(preseleccion_5, pedidos_de_la_prensa_5)
pedidos_de_la_prensa_7, preseleccion_7 = obtener_preseleccion_y_pedidos_de_la_prensa(path_7)
solucion_7 = solucion_para_Scaloni(preseleccion_7, pedidos_de_la_prensa_7)
pedidos_de_la_prensa_10_pocos, preseleccion_10_pocos = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_pocos)
solucion_10_pocos = solucion_para_Scaloni(preseleccion_10_pocos, pedidos_de_la_prensa_10_pocos)
pedidos_de_la_prensa_10_varios, preseleccion_10_varios = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_varios)
solucion_10_varios = solucion_para_Scaloni(preseleccion_10_varios, pedidos_de_la_prensa_10_varios)
pedidos_de_la_prensa_10_todos, preseleccion_10_todos = obtener_preseleccion_y_pedidos_de_la_prensa(path_10_todos)
solucion_10_todos = solucion_para_Scaloni(preseleccion_10_todos, pedidos_de_la_prensa_10_todos)


In [ ]:
resultados_esperados = obtener_resultados_esperados(resultados_sets_catedra)[:5]
pedidos_de_la_prensa = [pedidos_de_la_prensa_5, pedidos_de_la_prensa_7, pedidos_de_la_prensa_10_pocos,\
                        pedidos_de_la_prensa_10_varios, pedidos_de_la_prensa_10_todos]
resultados_obtenidos = [solucion_5, solucion_7, solucion_10_pocos,solucion_10_varios, solucion_10_todos]

In [ ]:
# Imprime los resultados obtenidos y los esperados en una tabla
resultados = []
i = 0
for resultado_esperado in resultados_esperados:
  es_correcto = "Sí" if resultado_esperado.cantidad_minima == len(resultados_obtenidos[i]) and verificar_solucion_para_Scaloni(pedidos_de_la_prensa[i], resultados_obtenidos[i]) else "No"
  resultados.append([resultado_esperado.nombre_archivo, resultado_esperado.cantidad_minima, len(resultados_obtenidos[i]), es_correcto]) 
  i += 1
print(tabulate(resultados, headers=['Archivo', 'Cantidad mínima esperada', 'Cantidad mínima obtenida', 'Se obtuvo el resultado esperado?']))

Archivo          Cantidad mínima esperada    Cantidad mínima obtenida  Se obtuvo el resultado esperado?
-------------  --------------------------  --------------------------  ----------------------------------
5.txt                                   2                           2  Sí
7.txt                                   2                           2  Sí
10_pocos.txt                            3                           3  Sí
10_varios.txt                           6                           6  Sí
10_todos.txt                           10                          10  Sí


##### 1 - Usando sets propios

In [27]:
pedidos_de_la_prensa_5_nuestro, preseleccion_5_nuestro = obtener_preseleccion_y_pedidos_de_la_prensa(path_5_nuestro)
solucion_5_nuestro = solucion_para_Scaloni(preseleccion_5_nuestro, pedidos_de_la_prensa_5_nuestro)
pedidos_de_la_prensa_15_nuestro, preseleccion_15_nuestro = obtener_preseleccion_y_pedidos_de_la_prensa(path_15_nuestro)
solucion_15_nuestro = solucion_para_Scaloni(preseleccion_15_nuestro, pedidos_de_la_prensa_15_nuestro)
pedidos_de_la_prensa_15_un_elemento_nuestro, preseleccion_15_un_elemento_nuestro = obtener_preseleccion_y_pedidos_de_la_prensa(path_15_un_elemento_nuestro)
solucion_15_un_elemento_nuestro = solucion_para_Scaloni(preseleccion_15_un_elemento_nuestro, pedidos_de_la_prensa_15_un_elemento_nuestro)
pedidos_de_la_prensa_20_nuestro, preseleccion_20_nuestro = obtener_preseleccion_y_pedidos_de_la_prensa(path_20_nuestro)
solucion_20_nuestro = solucion_para_Scaloni(preseleccion_20_nuestro, pedidos_de_la_prensa_20_nuestro)
pedidos_de_la_prensa_50_nuestro, preseleccion_50_nuestro = obtener_preseleccion_y_pedidos_de_la_prensa(path_50_nuestro)
solucion_50_nuestro = solucion_para_Scaloni(preseleccion_50_nuestro, pedidos_de_la_prensa_50_nuestro)

In [28]:
pedidos_de_la_prensa_nuestro = [pedidos_de_la_prensa_5_nuestro, pedidos_de_la_prensa_15_nuestro, pedidos_de_la_prensa_15_un_elemento_nuestro,\
                                pedidos_de_la_prensa_20_nuestro, pedidos_de_la_prensa_50_nuestro]
resultados_obtenidos_nuestros = [solucion_5_nuestro, solucion_15_nuestro, solucion_15_un_elemento_nuestro, solucion_20_nuestro, solucion_50_nuestro]
archivos = ['5.txt', '15.txt', '15_un_elemento.txt', '20.txt', '50.txt']

In [35]:
# Imprime los resultados obtenidos y los esperados en una tabla
resultados = []
i = 0
for resultado_obtenido in resultados_obtenidos_nuestros:
  es_correcto = "Sí" if verificar_solucion_para_Scaloni(pedidos_de_la_prensa_nuestro[i], resultado_obtenido) else "No"
  resultados.append([archivos[i], len(resultado_obtenido), es_correcto]) 
  i += 1
print(tabulate(resultados, headers=['Archivo', 'Cantidad minima obtenida', 'Es válida la soliución?']))

Archivo      Cantidad minima obtenida  Es válida la soliución?
---------  --------------------------  -------------------------
5.txt                               3  Sí
